# XGB model for Assessment Analysis

_Takes past scores and returns the predicted performance, current performance level and current growth trend_

In [ ]:
# Training notebook : https://www.kaggle.com/code/devil234/dynamic-assessment

### Usage from here-on out:

In [14]:
import joblib


average_score_model = joblib.load('../src/Weights/predicted_performance_level_model.pkl')
performance_level_model = joblib.load('../src/Weights/current_performance_level_model.pkl')
trend_model = joblib.load('../src/Weights/trend_model.pkl')
scaler = joblib.load('../src/Weights/scaler.pkl')

/home/gagan/Desktop/resume-projects/SmartEd/ML Service/.venv/lib/python3.10/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [24]:
import pandas as pd
import numpy as np

def extract_features(row):
    scores = row['score_history']
    if isinstance(scores, (int, float)):
        scores = [float(scores)]
    elif isinstance(scores, str):
        try:
            scores = ast.literal_eval(scores) if '[' in scores else [float(scores)]
        except (ValueError, SyntaxError):
            return [0, 0, 0]
    scores = [float(s) for s in scores if isinstance(s, (int, float)) or str(s).replace('.', '', 1).isdigit()]
    if not scores:
        return [0, 0, 0]
    return [np.mean(scores), np.var(scores), np.std(scores)]

In [36]:
scores = [100, 10, 10, 100]

sample_data = pd.DataFrame({
    'score_history': [scores]
})

sample_data[['mean_score', 'variance', 'std_dev']] = sample_data.apply(extract_features, axis=1, result_type='expand')

X_sample = sample_data[['mean_score', 'variance', 'std_dev']]

X_sample_scaled = scaler.transform(X_sample)


avg_score_pred = average_score_model.predict(X_sample_scaled)
perf_pred = performance_level_model.predict(X_sample_scaled)
trend_pred = trend_model.predict(X_sample_scaled)

print("Predicted Average Scores:", avg_score_pred)
print("Predicted Performance Levels:", perf_pred)
print("Predicted Trends:", trend_pred)

Predicted Average Scores: [54.958786]
Predicted Performance Levels: [1]
Predicted Trends: [2]


In [32]:
level_map = {'advanced': 0, 'intermediate': 1, 'beginner': 2}
trend_map = {'improving': 0, 'declining': 1, 'stable': 2}